In [1]:
# import all relevant packages
from tqdm.auto import tqdm
import praw
import time
import json
from datetime import datetime

client_id = "tVlTh68GbhKD1ygdzlB4-A"
client_secret = "CYmaD9keRNxENnCdZgOQAnszDn9UVg"
user_agent = "crimediscussion by u/capstone_crime"

reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [2]:
def download_subreddit_posts(subreddit_name, num_posts, keyword=None):
    subreddit = reddit.subreddit(subreddit_name)
    if keyword is None:
        return subreddit.new(limit=num_posts)
    else:
        return [post for post in subreddit.search(keyword, limit=num_posts)]

def download_post_comments(post, max_comments, comment_depth, keyword=None):
    post.comments.replace_more(limit=comment_depth)
    comments = post.comments.list()[:max_comments]

    if keyword is None:
        return comments
    else:
        return [comment for comment in comments if keyword.lower() in comment.body.lower()]

In [3]:
# creates a function to download subreddit data

def download_subreddit_data(subreddit_name, num_posts, max_comments=200, comment_depth=0, keyword=None):
    start_time = time.time()

    posts = download_subreddit_posts(subreddit_name, num_posts, keyword=keyword)

    subreddit_data = []
    for post in tqdm(posts, desc="Downloading posts", total=num_posts, unit="post"):
        post_data = {
            "title": post.title,
            "id": post.id,
            "url": post.url,
            "comments": []
        }

        comments = download_post_comments(post, max_comments, comment_depth)

        for comment in comments:
            comment_data = {
                "id": comment.id,
                "body": comment.body
            }
            post_data["comments"].append(comment_data)

        subreddit_data.append(post_data)

    print(f"Downloaded {num_posts} posts and their comments from r/{subreddit_name} in {time.time() - start_time:.2f} seconds")
    return subreddit_data

In [4]:
# function to save the data to a JSON file

def save_data_to_json(subreddit_data, output_filename):
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(subreddit_data, f, ensure_ascii=False, indent=4)

In [5]:
# function to run entire process

def main(subreddit_name, num_posts=1000, keyword=None):
    data = download_subreddit_data(subreddit_name, num_posts, keyword=keyword)
    output_filename = f"{subreddit_name}_data.json"
    save_data_to_json(data, output_filename)
    print(f"Data saved to {output_filename}")

In [6]:
if __name__ == "__main__":
    main('unresolvedmysteries',1000)

Downloaded 1000 posts and their comments from r/unresolvedmysteries in 1815.25 seconds
Data saved to unresolvedmysteries_data.json
